## ALS

 С предыдущей защиты, мне посоветовали также разобрать метод ALS. Начнем с него, продолжим с техникой валидации.

Если SVD - чисто алгоритмический подход, основывающийся на айген-декомпозиции квадратных матриц, то ALS является обучаемым методом, который соответственно имеет функцию потерь:
$$
\min_{X,Y}\sum_{r_{ui}\text{ observed}} (r_{ui} - x_u^Ty_i)^2 + \lambda(\sum_u \mid\mid x_u \mid\mid ^2 + \sum_i \mid\mid y_i \mid\mid^2)
$$

Здесь алгоритм собирает такие матрицы латентных представлений X и Y, которые бы минимизировали разницу между настоящими оценками пользователей и предсказываемыми скалярным произведением векторов пользователя и айтема. Также здесь присутствует регуляризация, которая штрафует очень большие нормы весов предметов и пользователей, похожая на стандартную L2 регуляризацию. Мы альтернируемся между фиксированием $x_u$ и $y_i$, чтобы превратить не-выпуклую np-hard задачу в выпуклую задачу оптимизации.

### Плюсы
- Можно задавать кол-во латентных признаков
- Можно распаралеллить (быстрые вычисления на больших данных, возможен онлайн)

### Минусы
- Лучше работает на implicit фидбеке (неявном), в нашем случае фидбек явный.
- Холодный старт
- Не берет во внимание последовательности

# Гипотезы для Финальной модели

Предполагаю, что стэккинг моделей может стать плодотворным подходом. Предлагаю такой подход:


1.   Matrix Factorization (ALS / SVD) -> Низкоранговое приближение
2.   Sequential Reccomender (Bert4Rec / LSTM4Rec / Gru4Rec) -> top k item-ов (взять из вероятностного распределения)
3.   Item-2-Item / User-2-User
4. Reranking (CatBoost / NN) -> rerank all candidates


# Валидация

In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install loguru polars -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 734.4 kB/s eta 0:00:00


In [3]:
def set_seed(seed: int = 42) -> None:
    import random
    import os

    np.random.seed(seed)
    random.seed(seed)
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

    return seed

set_seed()

# Load the Drive helper and mount
from google.colab import drive
drive_path = '/content/drive'
drive.mount(drive_path)

Random seed set as 42
Mounted at /content/drive


## Обучающая / Тренировочная выборка

In [4]:
df = pd.read_csv(drive_path + "/MyDrive/preprocessed_users_data.csv", index_col=0)
display(df.head())

,user_id,item_id,order_ts
0,550614,264,2023-01-01 00:28:09.000000
1,571051,580,2023-01-01 00:41:47.000000
2,571051,180,2023-01-01 00:41:47.000000
3,47164,5135,2023-01-01 00:53:35.000000
4,219072,2668,2023-01-01 01:02:29.000000


In [5]:
df["order_ts"] = pd.to_datetime(df["order_ts"])

Что делать с пользователями, которые совершили всего одну покупку? Оставлять эту покупку в трейне? Если идет разделение на train и test, нет пользы для тренировки модели, если потом нельзя провести предсказание.

In [ ]:
from typing import Tuple
from loguru import logger

def last_p_out(df: pd.DataFrame, p=1) -> Tuple[pd.DataFrame, pd.DataFrame]:
    assert len(df) != 0
    grouped = df.sort_values(by="order_ts").groupby("user_id")

    train_samples = []
    test_samples = []
    too_few_purchase_users_num = 0

    for user_id, user_interactions in grouped:
        # if user has only p purchases, delete it from dataset
        if user_interactions.shape[0] == p:
            too_few_purchase_users_num += 1
            continue
        assert user_interactions.shape[0] != 0, user_interactions

        train_sample = user_interactions.iloc[:-p]
        test_sample = user_interactions.iloc[-p:]

        train_samples.append(train_sample)
        test_samples.append(test_sample)
    train_data = pd.concat(train_samples)
    test_data = pd.concat(test_samples)

    logger.info(f"Number of unique users: {len(grouped)}")
    logger.info(f"{too_few_purchase_users_num} users had not enough of purchases to be in a dataset")

    return train_data, test_data


train, test = last_p_out(df, p=1)

In [ ]:
print(train.shape, '\n' , test.shape)
print("Ratio:", round(test.shape[0] / train.shape[0], 4), "is the portion of test data")

In [ ]:
train.to_csv("wb_train.csv")
train.head()

In [ ]:
test.to_csv("wb_test.csv")
test.head()

# Схема Валидации

Оптимальная схема валидации представляет из себя:
- Разделение на последовательности для каждого пользователя
- Разделение на train и val с помощью LeaveLastPOut (P=1)
- производим Кросс-Валидацию используя train:
    1.   Делаем K разделений на K фолдов, тестовый фолд - подмножество пользователей, у которых последнее взаимодействия из train скрыто. Пользователей, у которых изначально было всего 2 взаимодействия не берем.
    2.   Обучаем на такой кросс-валидации нашу модель, подбираем лучшие гипер параметры, основываемся на выбранной метрике (для гипер-параметров добавим GridSearch либо Optuna).
- Когда лучшие гипер-параметры выбраны, обучаем модель полностью на train
- Замеряем качество на test.

In [ ]:
def to_sequences(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    assert pd.api.types.is_datetime64_any_dtype(df['order_ts']), "Column is not of type datetime"

    df = df.sort_values(by="order_ts").drop(columns="order_ts")
    user_sequences  = df.groupby(by="user_id")["item_id"].apply(list)
    return user_sequences

In [8]:
from itertools import pairwise
from typing import Callable
from sklearn.model_selection import train_test_split

def cross_validate(user_sequences: pd.DataFrame, model_constructor: Callable, score: Callable, k=5, p=1):
    test_ratio = 1 / k

    scores = []
    for i in range(k):
        train_seq, test_seq = train_test_split(user_sequences, test_size=test_ratio, random_state=42)
        y_test = test_seq.apply(lambda lst: lst[-p])
        X_test = test_seq.apply(lambda lst: lst[:-p])

        model = model_constructor()
        model.fit(train_seq)
        # Тут мы ожидаем получить по sequence на каждого пользователя
        y_pred = model.predict(X_test)
        scores.append(score(y_test, y_pred))
    return scores / k

[1    1
 2    2
 dtype: int64,
 0    0
 dtype: int64]

Стоит иметь ввиду, что для candidate-search моделей нам нужны будут одни метрики (скорее всего классификационные, или регрессионные), а для ранжирующих моделей нужны будут ранжирующие метрики. Соответственно здесь может быть такой подход:
1. Обучить все candidate-search модели в изолированной среде и выбрать лучшие гипер-параметры для каждой (засчет CV)
2. Обучить на стэкинге этих моделей алгоритм из семейства Learning to Rank (например LambdaRank или CatBoost) и выбрать для него лучшие гипер-параметры (засчет CV)
3. Объединить стэкинг candidate-search моделей и ранжирующую модель.

# Обоснование Схемы Валидации

Здесь я опишу различные подходы и аспекты в условиях нашей проблематики.

Временное разделение: Поскольку у нас есть последовательные данные, подходящей стратегией будет временное разделение. Можно разделить данные на обучающий и тестовый наборы на основе времени, обеспечивая, чтобы тестовый набор следовал за обучающим. Это имитирует реальные сценарии, когда модель обучается на прошлых данных и тестируется на будущих взаимодействиях.

Sequential Models: При оценке последовательных моделей рекомендаций (например, LSTM4Rec, GRU4Rec) важно сохранить временной порядок взаимодействий. Можно разделить данные на последовательности и убедиться, что тестовый набор содержит последовательности, следующие за теми, которые находятся в обучающем наборе. Это гарантирует, что наша модель научится предсказывать будущие взаимодействия на основе прошлых.

Разделение на уровне пользователя: Учитывая низкое среднее количество покупок на пользователя, традиционное случайное разделение может не сработать хорошо, так как это может привести к разреженным обучающим данным для некоторых пользователей. Вместо этого можно рассмотреть разделение данных на уровне пользователя. Например, можно случайным образом выбрать часть пользователей для обучения и оставить остальных для тестирования. Это гарантирует, что наша модель увидит достаточное количество взаимодействий от каждого пользователя во время обучения.

Валидационный набор: Помимо разделения на обучающий и тестовый наборы, также стоит включить валидационный набор для настройки гиперпараметров и выбора модели. Учитывая возможное ограничение данных, отдельный валидационный набор поможет оценить производительность модели и ее обобщающую способность перед окончательным выбором модели.

Cross Validation: В зависимости от размера нашего набора данных и изменчивости поведения пользователей, можно также использовать техники перекрестной проверки (например, кросс-валидацию). Это поможет обеспечить устойчивость производительности модели на различных подмножествах данных.






# Sources:

- https://wiki.epfl.ch/edicpublic/documents/Candidacy%20exam/Evaluation.pdf
- https://www.kaggle.com/code/julian3833/h-m-implicit-als-model-0-014
- https://ods.ai/tracks/mts-recsys-df2020/blocks/cf056af1-ee00-4ed7-b8c9-988ee3ca8253
- https://en.wikipedia.org/wiki/Matrix_completion#Alternating_least_squares_minimization
- https://towardsdatascience.com/evaluation-metrics-for-recommendation-systems-an-overview-71290690ecba
- https://arxiv.org/pdf/2007.13237.pdf
- https://medium.com/the-owl/evaluating-recommender-systems-749570354976